<a href="https://colab.research.google.com/github/AmalBeldi/amal-beldi/blob/main/syst_recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install implicit==0.7.2 numpy pandas scipy scikit-learn tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os, zipfile, urllib.request, math, random
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import sparse
from scipy.stats import shapiro, ttest_rel, wilcoxon

from implicit.bpr import BayesianPersonalizedRanking

def set_all_seeds(seed: int):
    random.seed(seed)
    np.random.seed(seed)

def time_bin_from_timestamp(ts: int):
    # MovieLens timestamps are unix seconds; we use UTC hour for reproducibility.
    hour = pd.to_datetime(ts, unit="s", utc=True).hour
    # bins: [0,6), [6,12), [12,18), [18,24)
    if 0 <= hour < 6: return 0
    if 6 <= hour < 12: return 1
    if 12 <= hour < 18: return 2
    return 3

def bid_distribution(c_star: int, num_contexts: int, eta: float):
    if num_contexts <= 1:
        return np.array([1.0], dtype=float)
    p = np.full(num_contexts, eta / (num_contexts - 1), dtype=float)
    p[c_star] = 1.0 - eta
    p = p / p.sum()
    return p

def precision_at_k(ranked, gt_set, k):
    topk = ranked[:k]
    hits = sum(1 for i in topk if i in gt_set)
    return hits / k

def recall_at_k(ranked, gt_set, k):
    if len(gt_set) == 0:
        return 0.0
    topk = ranked[:k]
    hits = sum(1 for i in topk if i in gt_set)
    return hits / len(gt_set)

def ndcg_at_k(ranked, gt_set, k):
    topk = ranked[:k]
    dcg = 0.0
    for idx, item in enumerate(topk, start=1):
        if item in gt_set:
            dcg += 1.0 / math.log2(idx + 1)
    ideal_hits = min(len(gt_set), k)
    idcg = sum(1.0 / math.log2(i + 1) for i in range(1, ideal_hits + 1))
    return dcg / idcg if idcg > 0 else 0.0

def paired_test(x, y, alpha=0.05):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    deltas = y - x
    if len(deltas) < 3:
        return {"test": "n/a", "p_value": float("nan"), "cohen_d": float("nan"), "mean_delta": float(np.mean(deltas))}
    p_norm = shapiro(deltas).pvalue
    if p_norm >= alpha:
        stat, p = ttest_rel(y, x)
        test_name = "paired_t"
    else:
        stat, p = wilcoxon(y, x, zero_method="wilcox")
        test_name = "wilcoxon"
    cohen_d = float(np.mean(deltas) / (np.std(deltas, ddof=1) + 1e-12))
    return {"test": test_name, "p_value": float(p), "cohen_d": cohen_d, "mean_delta": float(np.mean(deltas))}


In [3]:
url = "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
os.makedirs("data", exist_ok=True)
zip_path = "data/ml-1m.zip"

if not os.path.exists(zip_path):
    urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall("data")

print("Extracted:", os.listdir("data/ml-1m")[:5])


Extracted: ['README', 'ratings.dat', 'users.dat', 'movies.dat']


In [4]:
ratings_path = "data/ml-1m/ratings.dat"
# ratings.dat format: UserID::MovieID::Rating::Timestamp
df = pd.read_csv(
    ratings_path, sep="::", engine="python",
    names=["user_raw", "item_raw", "rating", "timestamp"]
)

# implicit feedback: positive if rating >= 4
df = df[df["rating"] >= 4].copy()

# context: 4 time bins
df["context"] = df["timestamp"].apply(time_bin_from_timestamp)

# map ids to contiguous indices
user_ids = df["user_raw"].unique()
item_ids = df["item_raw"].unique()
user2idx = {u:i for i,u in enumerate(sorted(user_ids))}
item2idx = {m:i for i,m in enumerate(sorted(item_ids))}
df["user"] = df["user_raw"].map(user2idx)
df["item"] = df["item_raw"].map(item2idx)

num_users = df["user"].nunique()
num_items = df["item"].nunique()
num_contexts = df["context"].nunique()

print(df.head())
print("Users:", num_users, "Items:", num_items, "Contexts:", num_contexts)


   user_raw  item_raw  rating  timestamp  context  user  item
0         1      1193       5  978300760        3     0  1039
3         1      3408       4  978300275        3     0  3027
4         1      2355       5  978824291        3     0  2053
6         1      1287       5  978302039        3     0  1130
7         1      2804       5  978300719        3     0  2476
Users: 6038 Items: 3533 Contexts: 4


In [5]:
# Sort interactions by time
df = df.sort_values(["user", "timestamp"]).reset_index(drop=True)

# last interaction per user = test; others = train
last_idx = df.groupby("user").tail(1).index
test_df = df.loc[last_idx].copy()
train_df = df.drop(index=last_idx).copy()

print("Train interactions:", len(train_df))
print("Test interactions:", len(test_df))

# ground truth: one item per user (last)
gt_by_user = test_df.groupby("user")["item"].apply(lambda s: set(s.tolist())).to_dict()

# deterministic context for test point (c*)
cstar_by_user = test_df.set_index("user")["context"].to_dict()

# items seen in train (for filtering recommendations)
seen_by_user = train_df.groupby("user")["item"].apply(set).to_dict()


Train interactions: 569243
Test interactions: 6038


In [6]:
def build_user_item_matrix(df_part, num_users, num_items):
    # implicit library expects item-user matrix (items x users)
    rows = df_part["item"].to_numpy()
    cols = df_part["user"].to_numpy()
    data = np.ones(len(df_part), dtype=np.float32)
    mat = sparse.coo_matrix((data, (rows, cols)), shape=(num_items, num_users)).tocsr()
    return mat

def train_bpr(mat_items_users, seed=42, factors=64, iterations=50, lr=0.05, reg=1e-4):
    # implicit uses OpenMP threads; deterministic-ish with seed controlling init
    model = BayesianPersonalizedRanking(
        factors=factors, iterations=iterations,
        learning_rate=lr, regularization=reg,
        random_state=seed
    )
    model.fit(mat_items_users)
    return model

def train_models_per_context(train_df, num_users, num_items, num_contexts, seed=42,
                             factors=64, iterations=50, lr=0.05, reg=1e-4):
    mats = []
    models = []
    for c in range(num_contexts):
        df_c = train_df[train_df["context"] == c]
        mat_c = build_user_item_matrix(df_c, num_users, num_items)
        mats.append(mat_c)
        print(f"Context {c}: interactions={len(df_c)}")
        model_c = train_bpr(mat_c, seed=seed, factors=factors, iterations=iterations, lr=lr, reg=reg)
        models.append(model_c)
    return mats, models

# hyperparams (tu peux ajuster)
FACTORS = 64
EPOCHS = 50
LR = 0.05
REG = 1e-4


In [7]:
def score_all_items(model, user_index: int):
    # scores = item_factors dot user_factor
    uf = model.user_factors[user_index]  # shape [factors]
    scores = model.item_factors @ uf     # shape [num_items]
    return scores

def recommend_from_scores(scores, seen_items, topk):
    # filter seen items by setting -inf
    scores = scores.copy()
    if seen_items:
        scores[list(seen_items)] = -np.inf
    ranked = np.argpartition(-scores, range(topk))[:topk]
    ranked = ranked[np.argsort(-scores[ranked])]
    return ranked.tolist()

def evaluate(models, etas, Ks, seeds, trials_per_eta=1):
    """
    trials_per_eta: for deterministic baseline we don't need trials;
    for uncertainty modeling we can repeat stochastic draws, but here we use exact BID weights,
    so trials_per_eta can stay 1. (Keep for extension.)
    """
    all_results = []

    test_users = sorted(gt_by_user.keys())
    for seed in seeds:
        set_all_seeds(seed)
        print(f"\n=== Training for seed={seed} ===")
        mats, models_c = train_models_per_context(
            train_df, num_users, num_items, num_contexts,
            seed=seed, factors=FACTORS, iterations=EPOCHS, lr=LR, reg=REG
        )

        for eta in etas:
            # per-user metric arrays for paired tests
            per_user_metrics = {k: {"ndcg_det": [], "ndcg_unc": [],
                                    "prec_det": [], "prec_unc": [],
                                    "rec_det": [], "rec_unc": []}
                                for k in Ks}

            for u in tqdm(test_users, desc=f"Eval eta={eta}", leave=False):
                gt = gt_by_user[u]
                seen = seen_by_user.get(u, set())
                c_star = cstar_by_user[u]
                p = bid_distribution(c_star, num_contexts, eta)

                # deterministic scores from context c*
                scores_det = score_all_items(models_c[c_star], u)

                # uncertainty-aware expected scores: sum_c p(c)*scores_c
                scores_unc = np.zeros(num_items, dtype=np.float32)
                for c in range(num_contexts):
                    if p[c] == 0:
                        continue
                    scores_unc += p[c] * score_all_items(models_c[c], u)

                # rank once per K from the same score vectors
                maxK = max(Ks)
                ranked_det = recommend_from_scores(scores_det, seen, maxK)
                ranked_unc = recommend_from_scores(scores_unc, seen, maxK)

                for k in Ks:
                    per_user_metrics[k]["prec_det"].append(precision_at_k(ranked_det, gt, k))
                    per_user_metrics[k]["prec_unc"].append(precision_at_k(ranked_unc, gt, k))
                    per_user_metrics[k]["rec_det"].append(recall_at_k(ranked_det, gt, k))
                    per_user_metrics[k]["rec_unc"].append(recall_at_k(ranked_unc, gt, k))
                    per_user_metrics[k]["ndcg_det"].append(ndcg_at_k(ranked_det, gt, k))
                    per_user_metrics[k]["ndcg_unc"].append(ndcg_at_k(ranked_unc, gt, k))

            # aggregate results
            for k in Ks:
                nd_det = np.array(per_user_metrics[k]["ndcg_det"])
                nd_unc = np.array(per_user_metrics[k]["ndcg_unc"])
                pr_det = np.array(per_user_metrics[k]["prec_det"])
                pr_unc = np.array(per_user_metrics[k]["prec_unc"])
                rc_det = np.array(per_user_metrics[k]["rec_det"])
                rc_unc = np.array(per_user_metrics[k]["rec_unc"])

                test_out = paired_test(nd_det, nd_unc)

                all_results.append({
                    "seed": seed,
                    "eta": eta,
                    "K": k,
                    "ndcg_det_mean": float(nd_det.mean()),
                    "ndcg_unc_mean": float(nd_unc.mean()),
                    "prec_det_mean": float(pr_det.mean()),
                    "prec_unc_mean": float(pr_unc.mean()),
                    "rec_det_mean": float(rc_det.mean()),
                    "rec_unc_mean": float(rc_unc.mean()),
                    **test_out
                })

    return pd.DataFrame(all_results)

ETAS = [0.0, 0.1, 0.2, 0.3, 0.5]
KS = [5, 10, 20]
SEEDS = [1, 2, 3, 4, 5]


In [8]:
results_df = evaluate(models=None, etas=ETAS, Ks=KS, seeds=SEEDS)
results_df.head(10)



=== Training for seed=1 ===
Context 0: interactions=187661


  0%|          | 0/50 [00:00<?, ?it/s]

Context 1: interactions=63276


  0%|          | 0/50 [00:00<?, ?it/s]

Context 2: interactions=118664


  0%|          | 0/50 [00:00<?, ?it/s]

Context 3: interactions=199642


  0%|          | 0/50 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (3533,) (6038,) (3533,) 

In [9]:
# Summary over seeds for each eta and K
summary = (results_df
           .groupby(["eta", "K"])
           .agg(
               ndcg_det_mean=("ndcg_det_mean", "mean"),
               ndcg_unc_mean=("ndcg_unc_mean", "mean"),
               prec_det_mean=("prec_det_mean", "mean"),
               prec_unc_mean=("prec_unc_mean", "mean"),
               rec_det_mean=("rec_det_mean", "mean"),
               rec_unc_mean=("rec_unc_mean", "mean"),
               mean_delta=("mean_delta", "mean"),
               p_value=("p_value", "mean"),
               cohen_d=("cohen_d", "mean"),
           )
           .reset_index())

summary


NameError: name 'results_df' is not defined

In [10]:
from implicit.bpr import BayesianPersonalizedRanking
from scipy import sparse

def build_user_item_matrix(df_part, num_users, num_items):
    # user_items matrix: shape (users x items)
    rows = df_part["user"].to_numpy()
    cols = df_part["item"].to_numpy()
    data = np.ones(len(df_part), dtype=np.float32)
    mat = sparse.coo_matrix((data, (rows, cols)), shape=(num_users, num_items)).tocsr()
    return mat

def train_bpr(user_items, seed=42, factors=64, iterations=50, lr=0.05, reg=1e-4):
    model = BayesianPersonalizedRanking(
        factors=factors,
        iterations=iterations,
        learning_rate=lr,
        regularization=reg,
        random_state=seed
    )
    model.fit(user_items)
    return model

def train_models_per_context(train_df, num_users, num_items, num_contexts, seed=42,
                             factors=64, iterations=50, lr=0.05, reg=1e-4):
    models = []
    for c in range(num_contexts):
        df_c = train_df[train_df["context"] == c]
        user_items_c = build_user_item_matrix(df_c, num_users, num_items)
        print(f"Context {c}: interactions={len(df_c)}")
        model_c = train_bpr(user_items_c, seed=seed, factors=factors, iterations=iterations, lr=lr, reg=reg)
        models.append(model_c)
    return models


In [11]:
def score_all_items(model, user_index: int):
    uf = model.user_factors[user_index]          # [factors]
    scores = model.item_factors @ uf             # [num_items]
    return scores.astype(np.float32)


In [12]:
def evaluate(etas, Ks, seeds):
    all_results = []
    test_users = sorted(gt_by_user.keys())

    for seed in seeds:
        set_all_seeds(seed)
        print(f"\n=== Training for seed={seed} ===")
        models_c = train_models_per_context(
            train_df, num_users, num_items, num_contexts,
            seed=seed, factors=FACTORS, iterations=EPOCHS, lr=LR, reg=REG
        )

        for eta in etas:
            per_user = {k: {"nd_det": [], "nd_unc": [], "pr_det": [], "pr_unc": [], "rc_det": [], "rc_unc": []}
                        for k in Ks}

            for u in tqdm(test_users, desc=f"Eval eta={eta}", leave=False):
                gt = gt_by_user[u]
                seen = seen_by_user.get(u, set())
                c_star = cstar_by_user[u]
                p = bid_distribution(c_star, num_contexts, eta)

                scores_det = score_all_items(models_c[c_star], u)

                scores_unc = np.zeros(num_items, dtype=np.float32)
                for c in range(num_contexts):
                    if p[c] == 0:
                        continue
                    scores_unc += p[c] * score_all_items(models_c[c], u)

                maxK = max(Ks)
                ranked_det = recommend_from_scores(scores_det, seen, maxK)
                ranked_unc = recommend_from_scores(scores_unc, seen, maxK)

                for k in Ks:
                    per_user[k]["pr_det"].append(precision_at_k(ranked_det, gt, k))
                    per_user[k]["pr_unc"].append(precision_at_k(ranked_unc, gt, k))
                    per_user[k]["rc_det"].append(recall_at_k(ranked_det, gt, k))
                    per_user[k]["rc_unc"].append(recall_at_k(ranked_unc, gt, k))
                    per_user[k]["nd_det"].append(ndcg_at_k(ranked_det, gt, k))
                    per_user[k]["nd_unc"].append(ndcg_at_k(ranked_unc, gt, k))

            for k in Ks:
                nd_det = np.array(per_user[k]["nd_det"])
                nd_unc = np.array(per_user[k]["nd_unc"])
                pr_det = np.array(per_user[k]["pr_det"])
                pr_unc = np.array(per_user[k]["pr_unc"])
                rc_det = np.array(per_user[k]["rc_det"])
                rc_unc = np.array(per_user[k]["rc_unc"])

                test_out = paired_test(nd_det, nd_unc)

                all_results.append({
                    "seed": seed, "eta": eta, "K": k,
                    "ndcg_det_mean": float(nd_det.mean()),
                    "ndcg_unc_mean": float(nd_unc.mean()),
                    "prec_det_mean": float(pr_det.mean()),
                    "prec_unc_mean": float(pr_unc.mean()),
                    "rec_det_mean": float(rc_det.mean()),
                    "rec_unc_mean": float(rc_unc.mean()),
                    **test_out
                })

    return pd.DataFrame(all_results)

ETAS = [0.0, 0.1, 0.2, 0.3, 0.5]
KS = [5, 10, 20]
SEEDS = [1, 2, 3, 4, 5]

results_df = evaluate(ETAS, KS, SEEDS)
results_df.head()



=== Training for seed=1 ===
Context 0: interactions=187661


  0%|          | 0/50 [00:00<?, ?it/s]

Context 1: interactions=63276


  0%|          | 0/50 [00:00<?, ?it/s]

Context 2: interactions=118664


...

Context 2: interactions=118664


  0%|          | 0/50 [00:00<?, ?it/s]

Context 3: interactions=199642


  0%|          | 0/50 [00:00<?, ?it/s]

,seed,eta,K,ndcg_det_mean,ndcg_unc_mean,prec_det_mean,prec_unc_mean,rec_det_mean,rec_unc_mean,test,p_value,cohen_d,mean_delta
0,1,0.0,5,0.023407,0.023407,0.007122,0.007122,0.035608,0.035608,paired_t,NaN,0.000000,0.000000
1,1,0.0,10,0.031648,0.031648,0.006111,0.006111,0.061113,0.061113,paired_t,NaN,0.000000,0.000000
2,1,0.0,20,0.042025,0.042025,0.005134,0.005134,0.102683,0.102683,paired_t,NaN,0.000000,0.000000
3,1,0.1,5,0.023407,0.023743,0.007122,0.007221,0.035608,0.036105,wilcoxon,0.349211,0.011581,0.000336
4,1,0.1,10,0.031648,0.031710,0.006111,0.006078,0.061113,0.060782,wilcoxon,0.625268,0.002802,0.000062
